# Notebook 2: quantum circuits in cirq

In this notebook we'll be looking at quantum circuits with multiple qubits, and what kind of simulations we can do with them. 

## Multiple qubits

In the last notebook we only saw single qubit gates, but gates with multiple qubits are also very easy to implement in Cirq. For example, below we implement a `CNOT` gate surrounded by two `X` gates. As shown in fig. 4.11 in the book, this is equivalent to a controlled gate conditioned on the first qubit being in the state `0` instead of `1`, a fact we can also verify.

In [ ]:
import numpy as np

# Import cirq, install it if it's not installed.
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")
    import cirq

In [ ]:
# Create two qubits
qubit1 = cirq.NamedQubit("qubit1")
qubit2 = cirq.NamedQubit("qubit2")

# Create the cicruit step by step
circuit = cirq.Circuit()
circuit.append(cirq.X(qubit1))
circuit.append(cirq.CNOT(qubit1, qubit2))
circuit.append(cirq.X(qubit1))
print(circuit)

We get a little text-based schematic of the circuit. Note that the `CNOT` gate is displayed like `@ -- X`. The `@` denotes a control qubit, meaning that this is a controlled-`X` gate, but this is exactly what a `CNOT` gate is.

Let's simulate this circuit and see what it does:

In [ ]:
simulator = cirq.Simulator()
result = simulator.simulate(circuit)

print("\nOutput state:")
result.state_vector()

We get an output state of `[0,1,0,0]`, corresponding to the state $|01\rangle$. This is because if we don't specify anything, `cirq` assumes that all qubits are in the state $|0\rangle$. To change the input state, we can use the `initial_state` parameter as shown below. Here `0b01` represents the state $|01\rangle$, but this is completely equivalent to writing 1. You can convert from an integer `n` to a binary string using `bin(n)`.

We can convert from a vectorial representation to a bra-ket (Dirac) notation using `cirq.qis.dirac_noation`, or we can call the `diract_notation()` method of the simulator directly.


In [ ]:
result = simulator.simulate(circuit, initial_state=0b01)
print("Output state:", result.dirac_notation())

When creating multiple circuits, we can reuse the same qubits. The same `Simulator` object can also be used to simulate different circuits. For example, below we implement a simple circuit that uses three qubits, reusing the qubits `qubit1` and `qubit2` from the previous circuit.

In [ ]:
qubit3 = cirq.NamedQubit("qubit3")

other_circuit = cirq.Circuit()
other_circuit.append([cirq.X(qubit1), cirq.S(qubit2), cirq.X(qubit3)])
other_circuit.append([cirq.CNOT(qubit1,qubit2), cirq.H(qubit3)])
print(other_circuit)

simulator.simulate(other_circuit, initial_state=0b110).dirac_notation()

Note that here we use added three gates in parallel, since these three qubits act independently on different qubits. We can also use 3 separate calls to the `other_circuit.append` method to add gates in sequence, but this would use 3 lines of code instead of one. 

### Exercise 1
> Implement the two circuits shown in exercise 4.20 in the book. They are shown below as well. 
> The code below then simulates these two circuits for several several initial states and prints the results. Be sure to use the names `circuit1` and `circuit2` for the two circuits.


Circuit 1:  
```
qubit1: ───H───@───H───
               │
qubit2: ───H───X───H───
```

Circuit 2:  
```
qubit1: ───X───
           │
qubit2: ───@───
```

In [ ]:
# YOUR CODE HERE

print("\n"+"-"*20)
for state in range(4):
    res1 = simulator.simulate(circuit1, initial_state=state).dirac_notation()
    res2 = simulator.simulate(circuit2, initial_state=state).dirac_notation()
    print(f"Input: |{state:02b}>. Output1: {res1}, output2: {res2}")

## Moments in the circuit

To understand what a quantum circuit does, it is useful to see what happens to the input state at every moment (step) in the circuit. To do this, we can use the `simulate_moment_steps` method. This returns an iterator that yields a simulation of every moment in the circuit.

Let's see it in action for the circuit shown in figure 4.8, which implements the Toffoli gate using a controlled $V$-gate, where $V$ is such that $V^2=X$. The gate $V$ can be implemented in `cirq` by using `V = cirq.X**(1/2)`. To make a controlled version of a gate (conditioned on `n` qubits), we can call `V.controlled(n)`. The controlled $V^\dagger$ gate is then simply `controlled_V**(-1)`.

In [ ]:
circuit = cirq.Circuit()

V = cirq.X**(1/2)
controlled_V = V.controlled(1)
circuit.append([controlled_V(qubit2,qubit3)])
circuit.append([cirq.CNOT(qubit1, qubit2)])
circuit.append([(controlled_V**(-1))(qubit2,qubit3)])
circuit.append([cirq.CNOT(qubit1, qubit2)])
circuit.append([controlled_V(qubit1,qubit3)])

circuit

Now let's see what happens at every step in the circuit for the state $|110\rangle$. (Try changing the input state and see what happens.)

In [ ]:
step_num = 1
moments = simulator.simulate_moment_steps(circuit, initial_state=0b110)
for step in moments:
    print(f"State at moment t{step_num}: {step.dirac_notation()}")
    step_num += 1

### Exercise 2
> Implement the circuit shown in figure 4.9, which implements the Toffoli gate. It is shown below as well. Recall that we can for example create the gate $T^\dagger$ using `(cirq.T**-1)`.  
> Simulate what happens in the circuit at every moment using $|110\rangle$ as the input state.


Circuit3
```
qubit1: ──────────────────@──────────────────@───@──────────@───T───
                          │                  │   │          │
qubit2: ───────@──────────┼───────@───T^-1───┼───X───T^-1───X───S───
               │          │       │          │
qubit3: ───H───X───T^-1───X───T───X───T^-1───X───T───H──────────────
```

In [ ]:
# YOUR CODE HERE

## Universal quantum gates

This week we saw that we can approximate _any_ quantum circuit using just the $H$, $T$ (also called $\pi/8$-gate) and `CNOT` gate. This proof works in three stages:
1. Any circuit can be reduced to a sequence of _two-level_ operations
2. Any two-level operation can be implemented using `CNOT` and single qubit gates
3. Any single-qubit gate can be approximated using the $H$ and $T$ gates only.

In this part we'll be looking at the third part in some more detail. As first step recall that we can write _up to a phase_,
$$
    THTH = R_{\hat n}(\theta)
$$
where $\theta$ satisfies $\cos(\theta/2) = \cos(\pi/8)^2$, and $\hat n = (\cos \pi/8,\sin\pi/8,\cos\pi/8)$ (up to normalization). Let's first verify this numerically, and figure out what this phase is.

In [ ]:
from scipy.linalg import expm  # matrix exponential

H = cirq.H._unitary_()
T = cirq.T._unitary_()
mat1 = T @ H @ T @ H
print("THTH:")
print(mat1)

theta = 2 * np.arccos(np.cos(np.pi / 8) ** 2)
n_hat = np.array([np.cos(np.pi / 8), np.sin(np.pi / 8), np.cos(np.pi / 8)])
n_hat = n_hat / np.linalg.norm(n_hat)

n_hat_bloch = (
    n_hat[0] * cirq.X + n_hat[1] * cirq.Y + n_hat[2] * cirq.Z
)._unitary_()

# Rotation around the n_hat axis in Bloch sphere with angle theta
mat2 = expm(-0.5j * theta * n_hat_bloch)
print("\nR_n(theta):")
print(mat2)

print("\nmat1/mat2:")
print(mat1/mat2)


We confirm that indeed $THTH = e^{i\pi/4}R_{\hat n}(\theta)$ with the global phase factor $e^{i\pi/4}=\sqrt 2+i\sqrt 2$. 

Next, for any $\alpha$ and $\epsilon>0$ we can always find a $n$ such that
$$
    E(R_{\hat n}(\alpha),R_{\hat n}(\theta)^n) < \epsilon
$$

Let's verify this fact. For example, let $\alpha = 1$, then we can just enumerate values of $n$ until we an approximation that is good enough. Let's call the matrix $R_{\hat n}(\theta)=U$.

In [ ]:
# The matrix U
U = expm(-0.5j * theta * n_hat_bloch)

# The matrix we want to approximate
eps = 1e-4
alpha = 1.0
target = expm(-0.5j * alpha * n_hat_bloch)

# Try different values of n, until U**n is close enough to target
error = np.inf
n = 0
U_power = np.eye(2)  # Initialize with identity matrix
while error > eps:
    U_power = U_power @ U
    n += 1

    # np.linalg.norm(..., ord=np.inf) is the operator norm. It's not really
    # important which norm we choose, but this is what the book does.
    error = np.linalg.norm(U_power - target, ord=np.inf)

print("n =", n)
print("error:", error)

# Show the difference between U**n and target
np.linalg.matrix_power(U, n) - target

As we see it takes 55,939 steps to find a decent approximation! In the end the error is still $2.8\times 10^{-5}$, which is, depending on what we want to achieve, not necessarily a good approximation. The point here is that while we _can_ approximate $R_{\hat n}(\alpha)$ for any $\alpha$, it may be extremely expensive to do so. 


As shown in equation (4.81) in the book, we can approximate _any_ single qubit unitary gate using the following expression:
$$
    M(n_1,n_2,n_3) := R_{\hat n}(\theta)^{n_1}HR_{\hat n}(\theta)^{n_2}HR_{\hat n}(\theta)^{n_3}
$$

for $n_1,n_2,n_3$ positive integers. Let's see how we can use this fact in practice to approximate single qubit gates.

### Exercise 3a
> Implement a function `M(n1, n2, n3)` that returns the matrix corresponding to the unitary gate $M(n_1,n_2,n_3)$. It should pass the `assert` tests below. 

Be careful that `U**n` does _not_compute the matrix power of `U`, but rather raises each entry of `U` to the power `n`. Use `np.linalg.matrix_power(U, n)` instead.

In [ ]:
U = expm(-0.5j * theta * n_hat_bloch)
def M(n1, n2, n3):
    # YOUR CODE HERE
    pass


# Tests to verify your function is correct.
arr10_3_5 = np.array(
    [
        [-0.75773528 + 0.49766823j, -0.37152411 + 0.20033327j],
        [0.37152411 + 0.20033327j, -0.75773528 - 0.49766823j],
    ]
)
assert np.allclose(M(10, 3, 5), arr10_3_5)

arr7_8_9 = np.array(
    [
        [0.74170947 + 0.07254868j, -0.34625031 - 0.56983724j],
        [0.34625031 - 0.56983724j, 0.74170947 - 0.07254868j],
    ]
)
assert np.allclose(M(7, 8, 9), arr7_8_9)


### Exercise 3b
> By nested loops, iterate over all values of $n_1,n_2,n_3$ such that $n_1+n_2+n_3<30$ to find the indices for which $E(X, M(n_1,n_2,n_3))$ is minimal.

Hint: first figure out how to iterate over all values of $n_1,n_2,n_3$ such that $n_1+n_2+n_3=n$, for $n\geq 0$. Then use a while loop to iterate over all value 

In [191]:
X = np.array([[0, 1], [1, 0]])

# YOUR CODE HERE

We see that even the best approximation we found is really bad. We can increase $n$ to get a better approximation, but the quality of the approximation will improve extremely slowly. I invite you to change the matrix $X$ for a different unitary gate, and see how the error changes (hint: it will still be bad).

The bottom line is that universal approximations are a very useful theoretical device, but it does not immediately lead to practical approximations, even of very basic operations.